In [1]:
import pandas as pd
import numpy as np
import os
import shutil
import datetime
import glob
import warnings
warnings.filterwarnings("ignore")

In [2]:
data_dir = './data/'

input_dir = data_dir
output_dir = (data_dir + '_master_'
              + datetime.datetime.today().strftime('%Y%m%d') + '/'
              )
ipynb_name = 'data_master'

In [3]:
# create output folder
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.mkdir(output_dir)

In [4]:
def pick_notin(a, b):
    # a: pandas.core.series.Series
    # b: pandas.core.series.Series
    c = np.empty(0)
    for element in a.unique():
        if element not in b.unique(): c = np.append(c,element)
    return c

In [5]:
# set the path of the folder
path =  data_dir                
# advisable to use os.path.join as this makes concatenation OS independent
all_files = glob.glob(os.path.join(path, "*.csv"))

In [6]:
# only show the file names of csv files
source_name = os.path.basename(path).split('_')[0]
di_version = os.path.basename(path)
file_names = [os.path.basename(x).strip('_bsuy50') for x in all_files]

In [7]:
all_files

['./data/Network.csv',
 './data/ServiceArea.csv',
 './data/BusinessRules.csv',
 './data/Crosswalk2016.csv',
 './data/PlanAttributes.csv',
 './data/Crosswalk2015.csv',
 './data/Rate.csv',
 './data/BenefitsCostSharing.csv']

In [8]:
file_names

['Network.csv',
 'ServiceArea.csv',
 'BusinessRules.csv',
 'Crosswalk2016.csv',
 'PlanAttributes.csv',
 'Crosswalk2015.csv',
 'Rate.csv',
 'BenefitsCostSharing.csv']

In [ ]:
##This chunck is not necessary; we run this when we need to figure out which csv file is failed to be read
error_list = []
for f in all_files:
    try:
        df = pd.read_csv(f,sep='|')
    except Exception:
        print('Error in reading', f)
        error_list.append(f)

In [ ]:
# all_files.remove('/Users/yueqi/Desktop/work_core/standard_input_datasets_142/CHST08_a20220228_di/di20220301/CHST08_a20220228_di20220301/CL_SPHR.csv')

In [9]:
#show the length of row and col for each csv file
length_row = [len(pd.read_csv(f)) for f in all_files]
length_col = [len(pd.read_csv(f).columns) for f in all_files]

In [10]:
tab_source = pd.DataFrame(columns=['source_table_name', 'source_dataset_codename',
                                   'source_ncol','source_nrow','di_version',
                                   'di_action','description','notes','epic_definition'])
tab_source['source_table_name'] = file_names
tab_source['source_dataset_codename']=source_name # use the raw data folder name 
tab_source['di_version']=di_version # date for doing the deidentification, if raw data, show raw data
tab_source['source_ncol']=length_col
tab_source['source_nrow']=length_row
tab_source['description']=''
tab_source['di_action']=''
tab_source['notes']=''
tab_source['epic_definition']=''

In [11]:
master = pd.DataFrame(columns=['source_table_name', 'source_col_name','source_col_num','sparsity','example_value_1',
                              'example_value_2','example_value_3','example_value_4','example_value_5',
                              'di_action','di_status','description','epic_definition'])

for f in all_files:
    temp = pd.DataFrame(columns=['source_table_name', 'source_col_name','source_col_num','sparsity','example_value_1',
                                'example_value_2','example_value_3','example_value_4','example_value_5',
                                'di_action','di_status','description','epic_definition'])
    df_from_each_file = (pd.read_csv(f))
    temp['source_col_name'] = df_from_each_file.columns.values
    for i in temp['source_col_name']:
        list_5 = df_from_each_file[i].value_counts(dropna=False).keys()[:5].tolist()
        list_5[len(list_5):5] = [np.nan]* (5-len(list_5))
        temp.loc[temp['source_col_name']==i,'example_value_1'] = list_5[0] 
        temp.loc[temp['source_col_name']==i,'example_value_2'] = list_5[1] 
        temp.loc[temp['source_col_name']==i,'example_value_3'] = list_5[2]
        temp.loc[temp['source_col_name']==i,'example_value_4'] = list_5[3]
        temp.loc[temp['source_col_name']==i,'example_value_5'] = list_5[4]
        temp.loc[temp['source_col_name']==i,'sparsity'] = ((df_from_each_file[i].isnull().sum()) / 
                        (df_from_each_file[i].size)).tolist()
        temp.loc[temp['source_col_name']==i,'source_col_num'] = (df_from_each_file.columns.get_loc(i)) + 1
    temp['source_table_name'] = os.path.basename(f).strip('_bsuy50')
    master = master.append(temp,ignore_index = True, sort = False)
# convert sparsity column into percentage
master['sparsity'] = pd.Series(["{0:.2f}%".format(val * 100) for val in master['sparsity']], index = master.index)

In [12]:
with pd.ExcelWriter(output_dir+'data_master.xlsx') as writer:
    tab_source.to_excel(writer, sheet_name='source_tables',index = False,na_rep = 'NULL')
    master.to_excel(writer, sheet_name='source_variables',index = False,na_rep = 'NULL')